In [ ]:
# Instalação do Plotly
!pip install Plotly==4.12

# Instalação do Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

     |████████████████████████████████| 13.1MB 305kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 1.0MB 31.8MB/s 
     |████████████████████████████████| 3.5MB 30.9MB/s 
     |████████████████████████████████| 194kB 39.0MB/s 
     |████████████████████████████████| 1.8MB 35.7MB/s 
     |████████████████████████████████| 358kB 40.3MB/s 
  Created wheel for dash: filename=dash-1.19.0-cp37-none-any.whl size=84011 sha256=d5dbf0ed6716cff4776a39839aba27e4f78adbb04e5d5749d24de70817947f27
  Stored in directory: /root/.cache/pip/wheels/f7/50/a7/a230ff7f503b10120bff18c2524a375bb85a61ce6b519c8a77
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-cp37-none-any.whl size=1014870 sha256=9e0bbca690be8f787569054e6620faac9cd8d02c479bb98eb9436b55cb60daf3
  Stored in directory: /root/.cache/pip/wheels/46/a0/ec/2be2e8fc750e6

In [ ]:
#Importando os pacotes numpy, pandas e plotlyExpress
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
#Pacotes para criação de processos para suportar o serviço HTTP externo do Ngrok
import os.path
import sys
import json
import requests
import subprocess

# Pacotes para o utilizar as propriedades de conexões HTTP
from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

In [ ]:
# Definição da função de Download do Ngrok
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [ ]:
# Criação de um Túnel entre uma aplicação no Colab e uma URL externa no Ngrok
# Função para pegar a referência nesse túnel
Response = namedtuple('Response', ['url', 'error'])
def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

In [ ]:
download_ngrok()

--2021-03-25 16:02:45--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.206.108.108, 34.235.106.23, 3.211.152.205, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.206.108.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  24.3MB/s    in 0.5s    

2021-03-25 16:02:46 (24.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# Geração do programa exemplo_dash_1.py
%%writefile covid-vacine-progression-analysis.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

"""
  Aqui deverá ser feita a importação dos dataframes que serão base para a criação das figuras
"""
dataframe_pessoas_vacinadas_por_cent = pd.read_csv('https://raw.githubusercontent.com/Cogitus/covid-vacine-progression-analysis/main/data/pessoas_vacinadas_por_cent.csv')
dataframe_total_vacinacoes = pd.read_csv('https://raw.githubusercontent.com/Cogitus/covid-vacine-progression-analysis/main/data/total_vacinacoes.csv')
dataframe_vac_diarias_milhao = pd.read_csv('https://raw.githubusercontent.com/Cogitus/covid-vacine-progression-analysis/main/data/vac_diarias_milhao.csv')
dataframe_densidade_vacinas = pd.read_csv('https://raw.githubusercontent.com/Cogitus/covid-vacine-progression-analysis/main/data/densidade_vacinas.csv')

dataframe_densidade_vacinas = dataframe_densidade_vacinas.rename(columns={'Unnamed: 0': 'vacina'})
dataframe_densidade_vacinas = dataframe_densidade_vacinas.rename(columns={'num_paises': 'numero_de_paises'})

fig_pessoas_vacinadas = go.Figure(data=go.Choropleth(
    locations = dataframe_pessoas_vacinadas_por_cent['codigo_iso'],
    z = dataframe_pessoas_vacinadas_por_cent['pessoas_vacinadas_por_cent'],
    text = dataframe_pessoas_vacinadas_por_cent['pais'],
    colorscale = 'YlGnBu',
    autocolorscale=False,
    marker_line_width=0.5,
    colorbar_title = '% pessoas<br>vacinadas',
))

fig_pessoas_vacinadas.update_layout(
    title_text='Porcentagem de pessoas que tomaram pelo menos uma dose da vacina',
    geo=dict(
      showframe=False,
      showcoastlines=False,
      projection_type='equirectangular'
    )
)

fig_pessoas_vacinadas.data[0].update(zmin=0, zmax=60)

fig_total_doses = go.Figure(data=go.Choropleth(
    locations = dataframe_total_vacinacoes['codigo_iso'],
    z = dataframe_total_vacinacoes['total_vacinacoes'],
    text = dataframe_total_vacinacoes['pais'],
    colorscale = 'Blues',
    autocolorscale=False,
    marker_line_width=0.5,
    colorbar_title = 'Total<br>Vacinas<br>(milhões)'
))

fig_total_doses.update_layout(
    title_text='Total de doses aplicadas',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig_vac_diarias_milhao = go.Figure(data=go.Choropleth(
    locations = dataframe_vac_diarias_milhao['codigo_iso'],
    z = dataframe_vac_diarias_milhao['vacinacoes_diarias_por_milhao'],
    text = dataframe_vac_diarias_milhao['pais'],
    colorscale = 'YlGnBu',
    autocolorscale=False,
    reversescale=False,
    marker_line_width=0.5,
    colorbar_title = 'Vacinações<br>diárias<br>p/ milhão'
))

fig_vac_diarias_milhao.update_layout(
    title_text='Vacinações diárias por milhão',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig_vac_diarias_milhao.data[0].update(zmin=500, zmax=15000)

fig_disposicao_vacinas = fig = px.bar(dataframe_densidade_vacinas, x='numero_de_paises', y='vacina', title='Vacinas por número de países do Mundo', text='numero_de_paises')

app.layout = html.Div(children=[
    html.Title('Covid-19 World Vaccination Progression Analysis'),
    html.H1(children='Covid-19 World Vaccination Progression Analysis'),
    html.Div([
        html.P('Este é um projeto destinado à disciplina de ciência de dados'),
        html.P('Desenvolvido pelos alunos do curso de Engenharia de Computação da UFRN'),
        html.P('Ariel da Silva Alsina'),
        html.P('Jonatas Rodolfo Pereira dos Santos'),
        html.P('Luiz Paulo de Carvalho Alves')
    ]),
    html.Div([
      html.Div([
        dcc.Graph(
          id='pessoas-vacinadas-por-cent',
          figure=fig_pessoas_vacinadas
        )        
      ], style={'width': '50%', 'display': 'inline-block', 'padding': '0'}),
      html.Div([ 
        dcc.Graph(
            id='total-vacinacoes',
            figure=fig_total_doses
        )        
      ], style={'width': '50%', 'display': 'inline-block', 'padding': '0'}),
      html.Div([
        dcc.Graph(
          id='vacinacao-diaria-por-milhao',
          figure=fig_vac_diarias_milhao
        )        
      ], style={'width': '50%', 'display': 'inline-block', 'padding': '0'}),
      html.Div([
        dcc.Graph(
          id='disposicao_vacinas',
          figure=fig_disposicao_vacinas
        )        
      ], style={'width': '50%', 'display': 'inline-block', 'padding': '0'})
    ], style={'margin-top': '2em'}),
    html.Div([
      html.H2('10 Países com maior número absoluto de vacinações'),
      html.Img(src='https://raw.githubusercontent.com/Cogitus/covid-vacine-progression-analysis/main/img/top_10_countries.png', style={'width': '100%'}),
    ], style={'padding': '0', 'text-align': 'center', 'text-justify': 'inter-word', 'margin-top': '2em', 'margin-bottom': '2em'}),
    html.Div([
      html.H2('Agora, visualizando seu número relativo de vacinações'),
      html.Img(src='https://raw.githubusercontent.com/Cogitus/covid-vacine-progression-analysis/main/img/top_10_countries_percentage.png', style={'width': '100%'}),
    ], style={'padding': '0', 'text-align': 'center', 'text-justify': 'inter-word', 'margin-top': '2em', 'margin-bottom': '2em'}),
    html.Div([
      html.H2('Visualização dos Dados faltantes para cada país'),
      html.P('''Entende-se que países que existe um controle,
        organização e melhor abordagem  ao problema da vacinação em massa
        de sua população têm os dados condizentes a isto bem organizados,
        atualizados e acessíveis. Dito isso, a imagem a seguir,
        elucida visualmente os datasets de todos os países, em que as listras
        representam respectivamente (da primeira coluna mais à esquerda para a
        última coluna mais à direita): 1) País, 2) Código ISO do país, 3) Data
        da atualização diária, 3) Total de vacinações em números absolutos, 4)
        Número de pessoas com pelo menos uma dose de vacina aplicada, 5)
        Pessoas totalmente vacinadas (seja com uma, como com duas doses), 6)
        Número cru de vacinações diárias, 7) Número de vacinações diárias, 8)
        Total de vacinações à cada cem habitantes, 9) Porcentagem pessoas com 
        pelo menos um dose aplicada à cada cem habitantes, 10) Porcentagem de
        pessoas totalmente vacinadas à cada cem habitantes, 11) Porcentagem de
        vacinações diárias por milhão de habitantes, 12) Vacinas utilizadas no
        país, 13) Orgão responsável pela fonte dos dados, 14) Endereço Web onde
        os dados da fonte podem ser consultados. Faixas não contínuas
        representam dados faltantes nos bancos de dados; o mesmo vale para a
        ausência de colunas. Assim, é possível inferir, pela visualização das
        representações dos dados, como os países estão lidando com o processo
        de vacinação em termos de organização.
      ''', style={'margin': '1em', 'text-align': 'justify', 'text-justify': 'inter-word'}),
      html.Img(src='https://raw.githubusercontent.com/Cogitus/covid-vacine-progression-analysis/main/img/dados_faltantes.png', style={'width': '100%'}),        
    ], style={'width': '100%', 'text-align': 'center', 'text-justify': 'inter-word'}),
], style={'text-align': 'center', 'font-family': 'Lora, serif'})

if __name__ == '__main__':
    app.run_server(debug=True)

Writing covid-vacine-progression-analysis.py


In [ ]:
retorno = get_tunnel()
print(retorno)  # Para podermos saber o endereço da URL da Aplicação
!python covid-vacine-progression-analysis.py